In [1]:
#| default_exp map

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
import os
import re
import json
import subprocess
from pathlib import Path
import pandas as pd

In [4]:
#| export
def load(json_file):
    """
    Load JSON data from a file.

    **Args:**
    - `json_file` (`str` or `Path`): Path to the JSON file.

    **Returns:**
    - `data` (`dict`): Parsed JSON data.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    return data

In [5]:
#| export
def parse(filename):
    """
    Parse a filename to extract the video name and timestamp.

    Assumes filename format: `prefix_videoName_timestamp.extension`

    **Args:**
    - `filename` (`str`): Filename to parse.

    **Returns:**
    - `video_name` (`str`): Extracted video name without prefix.
    - `timestamp` (`float`): Extracted timestamp.
    """
    # Remove the file extension
    base = Path(filename).stem
    # Find the last underscore (before timestamp)
    last_underscore = base.rfind('_')
    if last_underscore == -1:
        raise ValueError(f"Filename '{filename}' does not contain an underscore.")
    
    timestamp_str = base[last_underscore + 1:]  # Get the part after the last underscore
    try:
        timestamp = float(timestamp_str)
    except ValueError:
        raise ValueError(f"Expected a timestamp after the last underscore in '{filename}', but got '{timestamp_str}'.")

    # Find the first underscore (after prefix)
    first_underscore = base.find('_')
    if first_underscore == -1:
        raise ValueError(f"Filename '{filename}' does not contain an underscore for prefix removal.")

    # Extract the main video name without the prefix
    video_name = base[first_underscore + 1:last_underscore]

    return video_name, timestamp

In [6]:
#| export
def extract(data, video_name, timestamp):
    """
    Extract labels for a given video and timestamp from the data.

    **Args:**
    - `data` (`list`): Annotations data.
    - `video_name` (`str`): Name of the video.
    - `timestamp` (`float`): Timestamp of the frame.

    **Returns:**
    - `labels` (`list`): List of labels corresponding to the timestamp.
    """
    labels = []
    for entry in data:
        file_upload = entry['file_upload']
        # Ignoring the prefix in the video names         
        prefix, video_file = file_upload.split('-', 1)
        if video_file == f"{video_name}.mp4":
            for annotation in entry['annotations']:
                for result in annotation['result']:
                    start, end = result['value']['start'], result['value']['end']
                    if start <= timestamp <= end:
                        labels.extend(result['value']['labels'])
    return labels

In [7]:
#| export
def map_labels(frames_folder, data):
    """
    Map labels to frames based on annotations.

    **Args:**
    - `frames_folder` (`str` or `Path`): Path to the folder containing frames.
    - `data` (`list`): Annotations data.

    **Returns:**
    - `results` (`list` of `dict`): List containing filename and corresponding labels.
    """
    frames_folder = Path(frames_folder)
    results = []
    # Retrieve all filenames ending with .png and sort them
    filenames = sorted(f.name for f in frames_folder.glob('*.png'))

    for filename in filenames:
        video_name, timestamp = parse(filename)
        labels = extract(data, video_name, timestamp)
        results.append({'filename': filename, 'timeline_label': ','.join(set(labels))})
    return results

In [8]:
#| export
def annotations(json_file):
    """
    Load annotations from a JSON file.

    **Args:**
    - `json_file` (`str` or `Path`): Path to the JSON file.

    **Returns:**
    - `annotations` (`list`): List of annotations.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
        
    annotations = []
    for entry in data:
        for annotation in entry['annotations']:
            for result in annotation['result']:
                labels = result['value']['labels']
                annotations.extend(labels)
    return annotations

In [9]:
#| export
def save(results, dataroot_path):
    """
    Save the mapping results to a CSV file.

    **Args:**
    - `results` (`list` of `dict`): Results from mapping labels to frames.
    - `data_root_path` (`str` or `Path`): Root path of the data directory.

    **Returns:**
    - `None`
    """
    df = pd.DataFrame(results)
    csv_path = Path(dataroot_path) / 'timeline_labels.csv'
    df.to_csv(csv_path, index=False)

In [ ]:
#| export
def main():
    """
    Main function to handle command-line arguments and execute the mapping and cutout processes.
    """
    import argparse

    parser = argparse.ArgumentParser(description="Map labels to frames and create video cutouts.")
    parser.add_argument('--annotations_file', type=str, required=True, help='Path to the annotations JSON file.')
    parser.add_argument('--frames_folder', type=str, required=True, help='Path to the folder containing frames.')
    parser.add_argument('--dataroot_path', type=str, required=True, help='Root path of the data directory.')
    parser.add_argument('--create_cutouts', action='store_true', help='Flag to create video cutouts.')
    parser.add_argument('--duration', type=int, default=5, help='Duration of the video cutouts in seconds.')

    args = parser.parse_args()

    # Load annotation data
    data = load(args.annotations_file)

    # Map labels to frames
    results = map_labels(args.frames_folder, data)

    # Save the results to CSV
    save(results, args.data_root_path)

    print("Timeline labels CSV file has been generated successfully.")

    # Create video cutouts if the flag is set
    if args.create_cutouts:
        labels_csv = Path(args.data_root_path) / 'timeline_labels.csv'
        cut(args.dataroot_path, labels_csv, duration=args.duration)
        print("Video cutouts have been created successfully.")

if __name__ == "__main__":
    main()